In [3]:
import numpy as np
from utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
from tqdm import tqdm
import random

In [4]:
ngram_range = 2
max_features = 20000
maxlen = 50
batch_size = 64
embedded_size = 128
epoch = 10

In [5]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [6]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [534, 7006, 1122, 535, 17, 3661, 7007, 812, 55, 178] ['adaptation', 'intricately', 'constructed', 'strange', 'way', 'nails', 'orleans', 'themes', 'without', 'true']


In [7]:
idx_trainset = []
for text in trainset.data:
    idx = []
    for t in text.split():
        try:
            idx.append(dictionary[t])
        except:
            pass
    idx_trainset.append(idx)

In [8]:
def create_ngram_set(input_list, ngram_value):
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))

def build_ngram(x_train):
    global max_features
    ngram_set = set()
    for input_list in tqdm(x_train, total=len(x_train), ncols=70):
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    max_features = np.max(list(indice_token.keys())) + 1
    return token_indice

def add_ngram(sequences, token_indice):
    new_sequences = []
    for input_list in tqdm(sequences, total=len(sequences), ncols=70):
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)
    return new_sequences

In [9]:
token_indice = build_ngram(idx_trainset)
X = add_ngram(idx_trainset, token_indice)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen)

100%|████████████████████████| 10662/10662 [00:00<00:00, 81235.23it/s]


In [10]:
X.shape

(10662, 50)

In [11]:
train_X, test_X, train_Y, test_Y = train_test_split(X, 
                                                    trainset.target,
                                                    test_size = 0.2)

In [12]:
class Model:
    def __init__(self, embedded_size, dict_size, dimension_output, learning_rate):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        self.logits = tf.layers.dense(tf.reduce_mean(encoder_embedded, 1), dimension_output)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=self.logits,
            labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(embedded_size,vocabulary_size+4,2,1e-3)
sess.run(tf.global_variables_initializer())

In [14]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : train_X[i:i+batch_size], model.Y : train_Y[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : test_X[i:i+batch_size], model.Y : test_Y[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.536932
time taken: 0.6459240913391113
epoch: 0, training loss: 0.689782, training acc: 0.533835, valid loss: 0.685098, valid acc: 0.536932

epoch: 1, pass acc: 0.536932, current acc: 0.572443
time taken: 0.3341085910797119
epoch: 1, training loss: 0.672378, training acc: 0.633576, valid loss: 0.672698, valid acc: 0.572443

epoch: 2, pass acc: 0.572443, current acc: 0.618845
time taken: 0.3254857063293457
epoch: 2, training loss: 0.647782, training acc: 0.716165, valid loss: 0.654821, valid acc: 0.618845

epoch: 3, pass acc: 0.618845, current acc: 0.664299
time taken: 0.367229700088501
epoch: 3, training loss: 0.611808, training acc: 0.773496, valid loss: 0.630317, valid acc: 0.664299

epoch: 4, pass acc: 0.664299, current acc: 0.691761
time taken: 0.3267521858215332
epoch: 4, training loss: 0.564377, training acc: 0.813910, valid loss: 0.601702, valid acc: 0.691761

epoch: 5, pass acc: 0.691761, current acc: 0.719223
time taken: 0.3157749176

In [15]:
logits = sess.run(model.logits, feed_dict={model.X:test_X})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

              precision    recall  f1-score   support

    negative       0.79      0.72      0.75      1076
    positive       0.74      0.81      0.77      1057

    accuracy                           0.76      2133
   macro avg       0.76      0.76      0.76      2133
weighted avg       0.76      0.76      0.76      2133

